In [1]:
import sys
sys.path.insert(1,"../AnalyseProfiles")
from Prepare_Events_Functions import *

In [2]:
import pandas as pd
import iris
import glob
import sys
import os
import time
import warnings
import pickle
from collections import OrderedDict

from Identify_Events_Functions import *
from Prepare_Data_Functions import *

pd.set_option('display.float_format', '{:.3f}'.format)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning, message=".*'+init=<authority>:<code>' syntax is deprecated.*")

######################################################
### Define data for finding the indepedent events at each gauge
######################################################

filtering_name = 'filtered_100'

### This is checking in which season the missing data is

In [18]:
missing_summer =0
missing_winter = 0

for yr in range(2006, 2021):
    # print(yr)
    ######################################################
    ### Get all the 5 minute data for one year, into one cube
    # (if it already exists in a pickle file, then load it from there)
    ######################################################
    general_filename = f'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/30mins/NIMROD_regridded_2.2km/{filtering_name}/AreaWeighted/{yr}/*'.format(year=yr)
    pickle_file_filepath = f"/nfs/a319/gy17m2a/PhD/datadir/cache/nimrod_30mins_2.2km/{filtering_name}/WholeYear/cube_{yr}.pkl"
    full_year_cube = load_cube_from_picklefile(pickle_file_filepath)
    
    times = full_year_cube.coord('time')
    time_resolution = 30
     # Make dataframe
    df = pd.DataFrame({'times': full_year_cube.coord('time').units.num2date(full_year_cube.coord('time').points),
                      'vals':range(0,full_year_cube.shape[0])})

    #############################################
    # Fill in missing values (e.g. some values just dont appear in the .nc files
    # Want these datetimes to still appear, but be recorded as np.nan
    #############################################
    df['times'] = pd.to_datetime([t.isoformat() for t in df['times']])

    # Determine the frequency
    freq = f'{time_resolution}T'  # 30 minutes

    # Create a full date range
    full_time_range = pd.date_range(start=df['times'].min(), end=df['times'].max(), freq=freq)

    # Set 'time' as index and reindex to the full range
    df.set_index('times', inplace=True)
    df = df.reindex(full_time_range)

    # Reset index and rename it back to 'time'
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'times'}, inplace=True)
    
    # df['season']=df['times'].apply(get_season)
    null_data = df[df.isnull().any(axis=1)].copy()
    null_data['times'] = null_data['times'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    null_data['season']=null_data['times'].apply(get_season)
    #print(np.unique(null_data['season'], return_counts=True))
    print(f"{yr}: {np.unique(null_data['season'], return_counts=True)[1][0]} summer and {np.unique(null_data['season'], return_counts=True)[1][1]} winter") 
    
    missing_summer = missing_summer+np.unique(null_data['season'], return_counts=True)[1][0]
    missing_winter = missing_winter+np.unique(null_data['season'], return_counts=True)[1][1]

2006: 602 summer and 975 winter
2007: 436 summer and 844 winter
2008: 333 summer and 776 winter
2009: 698 summer and 342 winter
2010: 1049 summer and 851 winter
2011: 29 summer and 415 winter
2012: 2260 summer and 871 winter
2013: 57 summer and 43 winter
2014: 79 summer and 4 winter
2015: 192 summer and 2831 winter
2016: 40 summer and 320 winter
2017: 40 summer and 342 winter
2018: 367 summer and 222 winter
2019: 287 summer and 88 winter
2020: 70 summer and 529 winter


In [16]:
# null_data[30:80]
# null_data[null_data['season']=='Summer'][:50]

In [68]:
missing_summer, missing_winter

(6539, 9453)

In [ ]:
### For UKCP18 data - none is missing - except from in first and last years

### For UKCP18 data, just testing whether the iris cube for each year has less data than expected

In [71]:
em='bb198'
yrs_range='2061_2081'
missing_summer =0
missing_winter = 0
for yr in range(2061, 2081):
    ######################################################
    ### Get all the 5 minute data for one year, into one cube
    # (if it already exists in a pickle file, then load it from there)
    ######################################################
    general_filename = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr{yr}*'
    pickle_file_filepath = f"/nfs/a319/gy17m2a/PhD/datadir/cache/UKCP18_30mins_{em}//WholeYear/cube_{yr}.pkl"

    full_year_cube = load_cube_from_picklefile(pickle_file_filepath)
    
    times = full_year_cube.coord('time')
    time_resolution = 30
     # Make dataframe
    df = pd.DataFrame({'times': full_year_cube.coord('time').units.num2date(full_year_cube.coord('time').points),
                      'vals':range(0,full_year_cube.shape[0])})
    
    
    null_data = df[df['vals'].isnull()].copy()
    print(yr, len(df))


2061 17280
2062 17280
2063 17280
2064 17280
2065 17280
2066 17280
2067 17280
2068 17280
2069 17280
2070 17280
2071 17280
2072 17280
2073 17280
2074 17280
2075 17280
2076 17280
2077 17280
2078 17280
2079 17280
2080 15840


In [54]:
yr=2080
######################################################
### Get all the 5 minute data for one year, into one cube
# (if it already exists in a pickle file, then load it from there)
######################################################
general_filename = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr{yr}*'
pickle_file_filepath = f"/nfs/a319/gy17m2a/PhD/datadir/cache/UKCP18_30mins_{em}//WholeYear/cube_{yr}.pkl"

full_year_cube = load_cube_from_picklefile(pickle_file_filepath)

times = full_year_cube.coord('time')
time_resolution = 30
 # Make dataframe
df = pd.DataFrame({'times': full_year_cube.coord('time').units.num2date(full_year_cube.coord('time').points),
                  'vals':range(0,full_year_cube.shape[0])})


null_data = df[df['vals'].isnull()].copy()
print(len(df))


times=df['times']

# Generate the expected range of times
start_time = min(times)
end_time = max(times)
expected_times = []

current_time = start_time
while current_time <= end_time:
    expected_times.append(current_time)
    current_time = current_time + datetime.timedelta(minutes=30)

# Find the missing times
missing_times = set(expected_times) - set(times)
missing_times
end_time

15840


cftime.Datetime360Day(2080, 11, 30, 23, 45, 0, 0, has_year_zero=True)

In [80]:
em='bc005'
yrs_range='2002_2020'
missing_summer =0
missing_winter = 0
for yr in range(2001, 2020):
    ######################################################
    ### Get all the 5 minute data for one year, into one cube
    # (if it already exists in a pickle file, then load it from there)
    ######################################################
#     general_filename = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr{yr}*'
    pickle_file_filepath = f"/nfs/a319/gy17m2a/PhD/datadir/cache/UKCP18_30mins_{em}//cube_{yr}.pkl"

    full_year_cube = load_cube_from_picklefile(pickle_file_filepath)
    
    times = full_year_cube.coord('time')
    time_resolution = 30
     # Make dataframe
    df = pd.DataFrame({'times': full_year_cube.coord('time').units.num2date(full_year_cube.coord('time').points),
                      'vals':range(0,full_year_cube.shape[0])})
    
    
    null_data = df[df['vals'].isnull()].copy()
    print(f"{yr}, {len(df)}")

2001, 17280
2002, 17280
2003, 17280
2004, 17280
2005, 17280
2006, 17280
2007, 17280
2008, 17280
2009, 17280
2010, 17280
2011, 17280
2012, 17280
2013, 17280
2014, 17280
2015, 17280
2016, 17280
2017, 17280
2018, 17280
2019, 17280


In [ ]:
### Finding the missing ones

In [45]:
times=df['times']

# Generate the expected range of times
start_time = min(times)
end_time = max(times)
expected_times = []

current_time = start_time
while current_time <= end_time:
    expected_times.append(current_time)
    current_time = current_time + datetime.timedelta(minutes=30)

# Find the missing times
missing_times = set(expected_times) - set(times)
missing_times


{cftime.Datetime360Day(2001, 4, 2, 11, 15, 0, 0, has_year_zero=True),
 cftime.Datetime360Day(2001, 4, 23, 3, 15, 0, 0, has_year_zero=True),
 cftime.Datetime360Day(2001, 4, 3, 15, 15, 0, 0, has_year_zero=True),
 cftime.Datetime360Day(2001, 4, 16, 8, 15, 0, 0, has_year_zero=True),
 cftime.Datetime360Day(2001, 4, 24, 3, 15, 0, 0, has_year_zero=True),
 cftime.Datetime360Day(2001, 4, 27, 5, 15, 0, 0, has_year_zero=True),
 cftime.Datetime360Day(2001, 4, 19, 1, 45, 0, 0, has_year_zero=True),
 cftime.Datetime360Day(2001, 4, 17, 22, 45, 0, 0, has_year_zero=True),
 cftime.Datetime360Day(2001, 4, 16, 6, 15, 0, 0, has_year_zero=True),
 cftime.Datetime360Day(2001, 4, 30, 22, 15, 0, 0, has_year_zero=True),
 cftime.Datetime360Day(2001, 4, 1, 22, 15, 0, 0, has_year_zero=True),
 cftime.Datetime360Day(2001, 4, 25, 18, 45, 0, 0, has_year_zero=True),
 cftime.Datetime360Day(2001, 4, 21, 4, 45, 0, 0, has_year_zero=True),
 cftime.Datetime360Day(2001, 4, 14, 3, 45, 0, 0, has_year_zero=True),
 cftime.Datetime3